In [1]:
%load_ext memory_profiler
import os
import tensorflow as tf
import warnings
import numpy as np
import pandas as pd
import scanpy as sc

warnings.simplefilter(action='ignore', category=FutureWarning)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

from scgamnn import scGAMNN
from utils import *
from metrics import *
warnings.filterwarnings("ignore")

sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi=100)
sc.settings.set_figure_params(dpi_save=300)
sc.logging.print_version_and_date()

adata1  = sc.read_csv("E:/scGAMNN/data/HSCs/Data1.csv")
adata2 = sc.read_csv("E:/scGAMNN/data/HSCs/Data2.csv")
adata = adata1.concatenate(adata2)
ident1 = pd.read_csv("E:/scGAMNN/data/HSCs/label1.csv")
ident2 = pd.read_csv("E:/scGAMNN/data/HSCs/label2.csv")
idents= np.vstack((ident1,ident2))
adata.obs['celltype'] = idents
match = np.array(pd.read_csv("E:/scGAMNN/data/HSCs/match.csv"))
sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e4)
sc.pp.log1p(adata)
adata.obs["batch"] = np.array([0] * len(ident1) + [1] * len(ident2))
sc.pp.highly_variable_genes(adata, n_top_genes=2000, flavor='seurat_v3',batch_key='batch')
adata = adata[:,adata.var.highly_variable]
sc.pp.scale(adata, max_value=10)
sc.tl.pca(adata, svd_solver='arpack')
adatas = [adata[adata.obs['batch'] == i,:].copy() for i in [0,1]]
scaledata = adata.X
adj,adj_n=construct_graph(adatas,match)

Running Scanpy 1.9.1, on 2023-07-13 15:41.
normalizing by total count per cell
    finished (0:00:00): normalized adata.X and added    'n_counts', counts per cell before normalization (adata.obs)
If you pass `n_top_genes`, all cutoffs are ignored.
extracting highly variable genes
--> added
    'highly_variable', boolean vector (adata.var)
    'highly_variable_rank', float vector (adata.var)
    'means', float vector (adata.var)
    'variances', float vector (adata.var)
    'variances_norm', float vector (adata.var)
computing PCA
    on highly variable genes
    with n_comps=50
    finished (0:00:01)


In [2]:
model = scGAMNN( scaledata,adj, adj_n, match=match,hidden_dim = 120, latent_dim = 50, decA = "DBL", layer_enc = "GCN")


In [3]:
import time
time_s = time.time()
%memit model.train(epochs=200, W_a=1, W_x=1, W_w=0.05)
print('time used:', time.time()-time_s)

Epoch 1  X_rec_loss: 0.800373   A_rec_loss: 0.14369944   MNN_loss: 0.14369944  total_loss:  1.0463367
Learning rate = 0.0020000
Epoch 2  X_rec_loss: 0.7996439   A_rec_loss: 0.01712969   MNN_loss: 0.01712969  total_loss:  0.9396362
Learning rate = 0.0020000
Epoch 3  X_rec_loss: 0.79583263   A_rec_loss: 0.0023525045   MNN_loss: 0.0023525045  total_loss:  0.9276368
Learning rate = 0.0020000
Epoch 4  X_rec_loss: 0.79122937   A_rec_loss: 0.002352531   MNN_loss: 0.002352531  total_loss:  0.926004
Learning rate = 0.0020000
Epoch 5  X_rec_loss: 0.7843881   A_rec_loss: 0.002352534   MNN_loss: 0.002352534  total_loss:  0.91580564
Learning rate = 0.0020000
Epoch 6  X_rec_loss: 0.77489835   A_rec_loss: 0.002352534   MNN_loss: 0.002352534  total_loss:  0.90462065
Learning rate = 0.0020000
Epoch 7  X_rec_loss: 0.7657301   A_rec_loss: 0.002352534   MNN_loss: 0.002352534  total_loss:  0.89600575
Learning rate = 0.0020000
Epoch 8  X_rec_loss: 0.7593028   A_rec_loss: 0.0023525346   MNN_loss: 0.002352534

Epoch 63  X_rec_loss: 0.67809343   A_rec_loss: 0.0025562847   MNN_loss: 0.0025562847  total_loss:  0.7111066
Learning rate = 0.0012800
Epoch 64  X_rec_loss: 0.68210846   A_rec_loss: 0.0025484336   MNN_loss: 0.0025484336  total_loss:  0.71322525
Learning rate = 0.0012800
Early stopping...
train Finish!
peak memory: 1280.01 MiB, increment: 455.95 MiB
time used: 56.34863066673279


In [4]:
import anndata as ad
adata_scGAMNN=ad.AnnData(model.embedding(scaledata, adj_n))
sc.pp.neighbors(adata_scGAMNN)
sc.tl.umap(adata_scGAMNN)
adata_scGAMNN.obs["batch"] = np.array(['Smart-seq2'] * len(ident1)+['MARS-seq'] * len(ident2))
adata_scGAMNN.obs["celltype"] =adata.obs["celltype"].to_list()


rep={'GMP':'0','CMP':'1','MEP':'2'}
times=[rep[i] if i in rep else i for i in pd.DataFrame(idents)[0]]
#列表中字符串转为数字
times=list(map(int,times))



computing neighbors
    using data matrix X directly
    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:09)
computing UMAP
    finished: added
    'X_umap', UMAP coordinates (adata.obsm) (0:00:06)


In [5]:
inte_scGAMNN=integrate_indicators (np.array(adata_scGAMNN.obsm['X_umap']),np.array(adata_scGAMNN.obs['batch']),np.array(adata_scGAMNN.obs['celltype']),np.array(adata_scGAMNN.obsm['X_umap']))
inte_scGAMNN_df=pd.DataFrame.from_dict(inte_scGAMNN,orient='index').T

[[10.2913265 -4.353655 ]
 [ 4.560591  -2.6001515]
 [ 7.376766  -4.860848 ]
 ...
 [ 1.3097748 -2.5454838]
 [ 8.475206   0.4563687]
 [-6.339938  -1.999099 ]] ASW_c: 0.6729 ASW_b: 0.7736 


In [6]:
adata1  = sc.read_csv("E:/scGAMNN/data/HSCs/Data1.csv")
adata2 = sc.read_csv("E:/scGAMNN/data/HSCs/Data2.csv")

import anndata as ad
adata_1in=ad.AnnData(adata_scGAMNN.X[0:813,],var=adata_scGAMNN.var)
adata_1in.obs['times'] = times[0:813]
sc.pp.neighbors(adata_1in)
sc.tl.diffmap(adata_1in)
root1=np.flatnonzero(adata_1in.obs['times'] == 1) [0] 
adata_1in.uns['iroot'] = root1
sc.tl.dpt(adata_1in)

adata1.obs['times'] = times[0:813]
sc.pp.normalize_per_cell(adata1)
sc.pp.log1p(adata1)
sc.pp.highly_variable_genes(adata1, n_top_genes=2000, flavor='seurat')
adata1 = adata1[:,adata1.var['highly_variable']]
sc.tl.pca(adata1, svd_solver='arpack')
sc.pp.neighbors(adata1)
sc.tl.diffmap(adata1)
adata1.uns['iroot'] = root1
sc.tl.dpt(adata1)

import scipy.stats as stats
tau1, p_value = stats.kendalltau(adata1.obs['dpt_pseudotime'], adata_1in.obs["dpt_pseudotime"])
print(tau1)
print(p_value)

computing neighbors
    using data matrix X directly
    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:00)
computing Diffusion Maps using n_comps=15(=n_dcs)
computing transitions
    finished (0:00:00)
    eigenvalues of transition matrix
    [1.         0.99391747 0.9851337  0.978026   0.95963407 0.9455159
     0.9379841  0.9264662  0.9112074  0.9034783  0.90009785 0.88951
     0.8745596  0.859845   0.85029197]
    finished: added
    'X_diffmap', diffmap coordinates (adata.obsm)
    'diffmap_evals', eigenvalues of transition matrix (adata.uns) (0:00:00)
computing Diffusion Pseudotime using n_dcs=10
    finished: added
    'dpt_pseudotime', the pseudotime (adata.obs) (0:00:00)
normalizing by total count per cell
    finished (0:00:00): normalized adata.X and added    'n_counts', counts per cell before normalization (adata.obs)
If you pass `n_top_genes`, all cutoffs ar

In [7]:
adata_2in=ad.AnnData(adata_scGAMNN.X[813:3543,],var=adata_scGAMNN.var)
adata_2in.obs['times'] = times[813:3543]
sc.pp.neighbors(adata_2in)
sc.tl.diffmap(adata_2in)
root2=np.flatnonzero(adata_2in.obs['times'] == 1) [0] 
adata_2in.uns['iroot'] = root2
sc.tl.dpt(adata_2in)


adata2.obs['times'] = times[813:3543]
sc.pp.normalize_per_cell(adata2)
sc.pp.log1p(adata2)
sc.pp.highly_variable_genes(adata2, n_top_genes=2000, flavor='seurat')
adata2 = adata2[:,adata2.var['highly_variable']]
sc.tl.pca(adata2, svd_solver='arpack')
sc.pp.neighbors(adata2)
sc.tl.diffmap(adata2)
adata2.uns['iroot'] = root2
sc.tl.dpt(adata2)

tau2, p_value = stats.kendalltau(adata2.obs['dpt_pseudotime'], adata_2in.obs["dpt_pseudotime"])
print(tau2)
print(p_value)

computing neighbors
    using data matrix X directly
    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:00)
computing Diffusion Maps using n_comps=15(=n_dcs)
computing transitions
    finished (0:00:00)
    eigenvalues of transition matrix
    [1.         0.9967179  0.9900288  0.9833275  0.98215723 0.9772958
     0.97139955 0.94856584 0.93774116 0.9236797  0.92039305 0.90931743
     0.9017304  0.89405185 0.8932249 ]
    finished: added
    'X_diffmap', diffmap coordinates (adata.obsm)
    'diffmap_evals', eigenvalues of transition matrix (adata.uns) (0:00:00)
computing Diffusion Pseudotime using n_dcs=10
    finished: added
    'dpt_pseudotime', the pseudotime (adata.obs) (0:00:00)
normalizing by total count per cell
    finished (0:00:00): normalized adata.X and added    'n_counts', counts per cell before normalization (adata.obs)
If you pass `n_top_genes`, all cutoffs

In [8]:
inte_scGAMNN_df.to_csv('result/inte_scGAMNN_HSCs.csv',index=0)
adata_scGAMNN.write('result/adata_scGAMNN_HSCs.h5ad')
# adata_scGAMNN=sc.read_h5ad('result/adata_scGAMNN_HSCs.h5ad')

In [9]:
tau_all=pd.DataFrame([tau1,tau2]).T
tau_all.to_csv('result/tau_scGAMNN_HSCs.csv',index=0)

